In [3]:
pip install opencv-python face_recognition numpy

Note: you may need to restart the kernel to use updated packages.


# Face Detection Web Application using Flask, OpenCV, and Face Recognition

This project is a Flask-based web application that allows users to upload images and videos for face detection. The application uses OpenCV and face_recognition libraries to identify and count faces in uploaded files.

In [ ]:
from flask import Flask, request, render_template_string
import cv2
import face_recognition
import os

app = Flask(__name__)

# HTML template for the upload form
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Face Detection Upload</title>
</head>
<body>
    <h1>Upload an Image or Video for Face Detection</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*,video/*" required>
        <input type="submit" value="Upload">
    </form>
    <div>{{ message }}</div>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE, message='')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return "No file part"
    
    file = request.files['file']
    
    if file.filename == '':
        return "No selected file"
    
    if file:
        # Save the uploaded file to a temporary location
        file_path = os.path.join('uploads', file.filename)
        file.save(file_path)

        # Determine if the uploaded file is an image or video
        if file_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            return detect_faces_in_image(file_path)
        elif file_path.lower().endswith(('.mp4', '.avi', '.mov')):
            return detect_faces_in_video(file_path)
        else:
            return "Unsupported file type. Please upload an image or video file."

def detect_faces_in_image(image_path):
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)
    return f"Number of faces detected in the image: {len(face_locations)}"

def detect_faces_in_video(video_path):
    video_capture = cv2.VideoCapture(video_path)
    face_count = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_frame)
        face_count += len(face_locations)

    video_capture.release()
    return f"Total number of faces detected in the video: {face_count}"

# Run the Flask app
if __name__ == "__main__":
    if not os.path.exists('uploads'):
        os.makedirs('uploads')
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Apr/2025 20:55:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2025 20:55:51] "POST /upload HTTP/1.1" 200 -
